In [1]:
from IPython.display import HTML
HTML('''<script>

code_show=true;

function code_toggle() {

if (code_show){

$('div.input').hide();

} else {

$('div.input').show();

}

code_show = !code_show

}

$( document ).ready(code_toggle);

</script>

<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import importlib
import clean_functions as cl
import geopy
import utm
%pylab inline
pd.set_option('display.max_column',None)

Populating the interactive namespace from numpy and matplotlib


In [3]:
os.getcwd();

## 1. Data gathering   
1) Activities files (from 2015 - 2019, Ayuntamiento Madrid)   
2) Madrid population database (1st january 2019, Ayuntamiento Madrid)   
3) Madrid floating population (16-22 april 2018, Private source)   

### 1) Activities file: 
Data is downloaded from madrid.es open data portal: https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=23160329ff639410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default   

We load the files into memory as a pandas dataframe. Files are in '.csv' format and uncompressed.   

I found some problems while loading the files due to incorrect separators parsing so I did some cleaning through Command Line to remove "ambigous separators" (I generated "_clean" versions of the files).

Encoding is Latin9 (although in documentation UTF8 is mentioned).      

After checking all files available in the web portal (premises, licences and activities (== epigrafes in Spanish), I will work over "epigrafes" file that contains all the premises information plus the status.   

I will do the study on a yearly basis. Since 2019 data is only available till Sep'19, I will use the September files for all the other years.  

Notes: I decided not use 2014 file since doesn't have the same fields as the others and goes too far in time.

In [4]:
# 2019 data. A clean version is needed to correct wrong separators via command line
df_epi19 = pd.read_csv('Data/censolocales/OPEN DATA Locales-Epigrafes201909_clean.csv',sep=';',encoding='latin9',low_memory=False)
# 2018 data. A clean version is needed to correct wrong separators via command line
df_epi18 = pd.read_csv('Data/censolocales/OPEN DATA Locales-Epigrafes201809_clean.csv',sep=';',encoding='latin9',low_memory=False)
# 2017 data. September file was corrupted (many NaN). I will use Nov. file instead
df_epi17 = pd.read_csv('Data/censolocales/OPEN DATA Locales-Epigrafes201711_clean.csv',sep=';',encoding='latin9',low_memory=False)
# 2016 data. This file is ok, no clean version is needed.
df_epi16 = pd.read_csv('Data/censolocales/OPEN DATA Locales-Epigrafes201609.csv',sep=';',encoding='latin9',low_memory=False)
# 2015 data. This file is ok, no clean version is needed.
df_epi15 = pd.read_csv('Data/censolocales/OPEN DATA Locales-Epigrafes201509.csv',sep=';',encoding='latin9',low_memory=False)

Each new year file contains the commercial premises of previous years plus the new opened that year. All of the files have to have the same columns. I check it and it's correct: 2019 file has more rows than any and all of teh files have 47 columns

In [5]:
print('2019 commercial premises file shape:', df_epi19.shape)
print('2018 commercial premises file shape:', df_epi18.shape)
print('2017 commercial premises file shape:', df_epi17.shape)
print('2016 commercial premises file shape:', df_epi16.shape)
print('2015 commercial premises file shape:', df_epi15.shape)

2019 commercial premises file shape: (163355, 46)
2018 commercial premises file shape: (162467, 46)
2017 commercial premises file shape: (161097, 46)
2016 commercial premises file shape: (159604, 46)
2015 commercial premises file shape: (157206, 46)


In [6]:
print('Commercial premises file information:')
df_epi19.columns

Commercial premises file information:


Index(['id_local', 'id_distrito_local', 'desc_distrito_local',
       'id_barrio_local', 'desc_barrio_local', 'cod_barrio_local',
       'id_seccion_censal_local', 'desc_seccion_censal_local',
       'coordenada_x_local', 'coordenada_y_local', 'id_tipo_acceso_local',
       'desc_tipo_acceso_local', 'id_situacion_local', 'desc_situacion_local',
       'id_vial_edificio', 'clase_vial_edificio', 'desc_vial_edificio',
       'id_ndp_edificio', 'id_clase_ndp_edificio', 'nom_edificio',
       'num_edificio', 'cal_edificio', 'secuencial_local_PC', 'id_vial_acceso',
       'clase_vial_acceso', 'desc_vial_acceso', 'id_ndp_acceso',
       'id_clase_ndp_acceso', 'nom_acceso', 'num_acceso', 'cal_acceso',
       'coordenada_x_agrupacion', 'coordenada_y_agrup', 'id_agrupacion',
       'nombre_agrupacion', 'id_tipo_agrup', 'desc_tipo_agrup',
       'id_planta_agrupado', 'id_local_agrupado', 'rotulo', 'id_seccion',
       'desc_seccion', 'id_division', 'desc_division', 'id_epigrafe',
       'desc_epi

In [7]:
print('Commercial premises file head (2 first rows):')
df_epi19.head(2)

Commercial premises file head (2 first rows):


,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local,id_seccion_censal_local,desc_seccion_censal_local,coordenada_x_local,coordenada_y_local,id_tipo_acceso_local,desc_tipo_acceso_local,id_situacion_local,desc_situacion_local,id_vial_edificio,clase_vial_edificio,desc_vial_edificio,id_ndp_edificio,id_clase_ndp_edificio,nom_edificio,num_edificio,cal_edificio,secuencial_local_PC,id_vial_acceso,clase_vial_acceso,desc_vial_acceso,id_ndp_acceso,id_clase_ndp_acceso,nom_acceso,num_acceso,cal_acceso,coordenada_x_agrupacion,coordenada_y_agrup,id_agrupacion,nombre_agrupacion,id_tipo_agrup,desc_tipo_agrup,id_planta_agrupado,id_local_agrupado,rotulo,id_seccion,desc_seccion,id_division,desc_division,id_epigrafe,desc_epigrafe
0,270440895,3,RETIRO,304,IBIZA,NaN,NaN,55,0,0,NaN,Agrupado,NaN,Abierto,370400,CALLE,IBIZA ...,11012388,1,NUM,8,,0,NaN,CALLE,IBIZA ...,11012388,1,NUM,8,,"442533,58","4474389,51",99000191.0,MERCADO MUNICIPAL DE IBIZA,12.0,Mercado Municipal,PB,7,BAR HERMANOS BENAYAS,I,HOSTELERIA,56,SERVICIOS DE COMIDAS Y BEBIDAS,563005,BAR SIN COCINA
1,270440899,5,CHAMARTIN,502,PROSPERIDAD,NaN,NaN,23,"443460,59","4477604,5",NaN,Puerta Calle,NaN,Abierto,210200,CALLE,CORAZON DE MARIA ...,20138306,1,NUM,57,,20,NaN,CALLE,CORAZON DE MARIA ...,20138306,1,NUM,57,,NaN,NaN,NaN,NaN,NaN,NaN,PB,NaN,LA PARRILLA BERENGUER,I,HOSTELERIA,56,SERVICIOS DE COMIDAS Y BEBIDAS,561005,BAR CON COCINA


The files have more more rows than unique id_locals. This is because the same local can be licenced with one or more activities (or "epigrafes").

In [8]:
print('2019 commercial premises unique id_locals: % 0.0f' %(len(df_epi19['id_local'].unique())))
print('2018 commercial premises unique id_locals: % 0.0f' %(len(df_epi18['id_local'].unique())))
print('2017 commercial premises unique id_locals: % 0.0f' %(len(df_epi17['id_local'].unique())))
print('2016 commercial premises unique id_locals: % 0.0f' %(len(df_epi16['id_local'].unique())))
print('2015 commercial premises unique id_locals: % 0.0f' %(len(df_epi15['id_local'].unique())))

2019 commercial premises unique id_locals:  147344
2018 commercial premises unique id_locals:  146607
2017 commercial premises unique id_locals:  145752
2016 commercial premises unique id_locals:  144920
2015 commercial premises unique id_locals:  144048


I check the % of No activity premises vs total (note, I have not removed duplicated "id_locals" so it is a gross calculation). We can observe a descending trend.

##### 2019

In [9]:
print('Number of locals with No activity in 2019: % 0.0f' % (df_epi19[df_epi19.desc_epigrafe.isnull()].groupby('rotulo').id_local.count().values[0]))
print('perc of total: % 0.2f' % (df_epi19[df_epi19.desc_epigrafe.isnull()].groupby('rotulo').id_local.count().values[0]/len(df_epi19)))

Number of locals with No activity in 2019:  41276
perc of total:  0.25


##### 2018

In [10]:
print('Number of locals with No activity in 2018: % 0.0f' % (df_epi18[df_epi18.desc_epigrafe.isnull()].groupby('rotulo').id_local.count().values[0]))
print('perc of total: % 0.2f' % (df_epi18[df_epi18.desc_epigrafe.isnull()].groupby('rotulo').id_local.count().values[0]/len(df_epi18)))

Number of locals with No activity in 2018:  42051
perc of total:  0.26


##### 2017

In [11]:
print('Number of locals with No activity in 2017: % 0.0f' % (df_epi17[df_epi17.desc_epigrafe.isnull()].groupby('rotulo').id_local.count().values[0]))
print('perc of total: % 0.2f' % (df_epi17[df_epi17.desc_epigrafe.isnull()].groupby('rotulo').id_local.count().values[0]/len(df_epi17)))

Number of locals with No activity in 2017:  42527
perc of total:  0.26


##### 2016

In [12]:
print('Number of locals with No activity in 2016: % 0.0f' % (df_epi16[df_epi16.desc_epigrafe.isnull()].groupby('rotulo').id_local.count().values[0]))
print('perc of total: % 0.2f' % (df_epi16[df_epi16.desc_epigrafe.isnull()].groupby('rotulo').id_local.count().values[0]/len(df_epi16)))

Number of locals with No activity in 2016:  43549
perc of total:  0.27


##### 2015

In [13]:
print('Number of locals with No activity in 2015: % 0.0f' % (df_epi15[df_epi15.desc_epigrafe.isnull()].groupby('rotulo').id_local.count().values[0]))
print('perc of total: % 0.2f' % (df_epi15[df_epi15.desc_epigrafe.isnull()].groupby('rotulo').id_local.count().values[0]/len(df_epi15)))

Number of locals with No activity in 2015:  44915
perc of total:  0.29


### 2) Madrid Population database
Data from madrid.es open data portal: https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=1d755cde99be2410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default

Excel format. I combine information from diferent census files in the excel for the study

In [14]:
#Madrid population main KPIs
df1 = pd.read_excel('Data/censopob/censo_Madrid_distrito.xlsx')
print('Madrid population main KPIs per district:')
df1.info()

Madrid population main KPIs per district:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 35 columns):
id_distrito_local                                                         21 non-null int64
distrito                                                                  21 non-null object
Población                                                                 21 non-null int64
Hombre                                                                    21 non-null int64
Mujeres                                                                   21 non-null int64
Densidad (Habitantes / Ha.)                                               21 non-null float64
Edad promedio                                                             21 non-null float64
Edad mediana                                                              21 non-null float64
Renta media/persona (euros)                                               21 non-null int64
Total Hogares       

### 3) Madrid floating population
Info from Kineo. Floating population in Madrid Districts from 14-22 of april 2018

Excel format

In [15]:
# Floating_population
df2 = pd.read_excel('Data/PF/PF_diaria_norm.xlsx')
print('Madrid one week floating population per district:')
df2.dtypes

Madrid one week floating population per district:


id_distrito_local                      int64
España fuera barrio dia laboral        int64
Extranjero fuera barrio dia laboral    int64
Total fuera barrio dia laboral         int64
España fuera barrio fin semana         int64
Extranjero fuera barrio fin semana     int64
Total fuera barrio fin semana          int64
Total barrio dia laboral               int64
Total barrio fin semana                int64
Total trabajo dia laboral              int64
Total trabajo fin semana               int64
dtype: object

## 2. Data preparation
1) Commercial premises status normalization   
2) NaN treatment   
3) Merge all years commercial premises info in a single DataFrame   
4) Generate target variable   
5) Standardize type of activities   
6) Generate interesting variables and convert UTM coordinates
7) Add population info   
8) Select activities for analysis   
9) Merge with info points in radius
10) Export to csv

I have created a modules script **clean_functions.py** with some functions to "simplify" the notebook

In [16]:
import importlib
import clean_functions as cl
importlib.reload(cl)

<module 'clean_functions' from '/home/dsc/Python_notebooks/TFM/TBV/TBV_v1/clean_functions.py'>

### 1) Status normalization

I create a new column with function **cl.new_col**: 'conc': concat('rotulo','desc_vial_acceso') to identify unique commercial premises. One "id_local" can have different "rotulo" values but with the address (des_vial_acceso), I have unique values. I will use it to merge all the files. I check the new columns is already created

In [17]:
df_epi19 = cl.new_col(df_epi19)
df_epi18 = cl.new_col(df_epi18)
df_epi17 = cl.new_col(df_epi17)
df_epi16 = cl.new_col(df_epi16)
df_epi15 = cl.new_col(df_epi15)

In [18]:
df_epi16.columns.values

array(['id_local', 'id_distrito_local', 'desc_distrito_local',
       'id_barrio_local', 'desc_barrio_local', 'cod_barrio_local',
       'id_seccion_censal_local', 'desc_seccion_censal_local',
       'coordenada_x_local', 'coordenada_y_local', 'id_tipo_acceso_local',
       'desc_tipo_acceso_local', 'id_situacion_local',
       'desc_situacion_local', 'id_vial_edificio', 'clase_vial_edificio',
       'desc_vial_edificio', 'id_ndp_edificio', 'id_clase_ndp_edificio',
       'nom_edificio', 'num_edificio', 'cal_edificio',
       'secuencial_local_PC', 'id_vial_acceso', 'clase_vial_acceso',
       'desc_vial_acceso', 'id_ndp_acceso', 'id_clase_ndp_acceso',
       'nom_acceso', 'num_acceso', 'cal_acceso',
       'coordenada_x_agrupacion', 'coordenada_y_agrup', 'id_agrupacion',
       'nombre_agrupacion', 'id_tipo_agrup', 'desc_tipo_agrup',
       'id_planta_agrupado', 'id_local_agrupado', 'rotulo', 'id_seccion',
       'desc_seccion', 'id_division', 'desc_division', 'id_epigrafe',
       'd

In [19]:
#DataFrames copy to preserve info
df_epi19_cl = df_epi19.copy()
df_epi18_cl = df_epi18.copy()
df_epi17_cl = df_epi17.copy()
df_epi16_cl = df_epi16.copy()
df_epi15_cl = df_epi15.copy()

With **cl.estado** function, I unify all status (desc_situacion_local) containing 'Baja' or 'LOCAL SIN ACTIVIDAD' in the status 'Cerrado'. I create a new column 'desc_sit_loc_modif'. I do this for all DataFrames.

In [20]:
print('Before the status value unification:')
df_epi19_cl['desc_situacion_local'].value_counts()

Before the status value unification:


Abierto               117830
Cerrado                34738
Uso vivienda            6473
Baja Reunificacion      2444
Baja                    1018
En obras                 841
Baja PC Asociado          11
Name: desc_situacion_local, dtype: int64

In [21]:
df_epi19_cl['desc_sit_loc_modif'] = cl.estado(df_epi19_cl['desc_epigrafe'],df_epi19_cl['desc_situacion_local'])
df_epi18_cl['desc_sit_loc_modif'] = cl.estado(df_epi18_cl['desc_epigrafe'],df_epi18_cl['desc_situacion_local'])
df_epi17_cl['desc_sit_loc_modif'] = cl.estado(df_epi17_cl['desc_epigrafe'],df_epi17_cl['desc_situacion_local'])
df_epi16_cl['desc_sit_loc_modif'] = cl.estado(df_epi16_cl['desc_epigrafe'],df_epi16_cl['desc_situacion_local'])
df_epi15_cl['desc_sit_loc_modif'] = cl.estado(df_epi15_cl['desc_epigrafe'],df_epi15_cl['desc_situacion_local'])
print('After the status value unification:')
df_epi19_cl['desc_sit_loc_modif'].value_counts()

After the status value unification:


Abierto         116704
Cerrado          39337
Uso vivienda      6473
En obras           841
Name: desc_sit_loc_modif, dtype: int64

### 2) NaN treatment

With module **cl.na** I fillna according to Epigrafes nomenclature (https://datos.madrid.es/FWProjects/egob/Catalogo/Economia/Ficheros/Estructura_DS_FicheroCLA.pdf):   
- Locales with 'rotulo' == 'SIN ACTIVIDAD'.
- I regularize the locals that do not belong to a group ('Agrupacion')   
- I fill in the column id_situation local

I do this for 2019 DataFrame since it is going to be the baseline to merge with the status of the previous years

In [22]:
df_epi19_cl_d = cl.na(df_epi19_cl)

### 3) Merge commercial premises info in one single DataFrame

With function **cl.col_rest**, I drop most of the columns 2015-2018 DataFrames and just keep columns_of_interest=['id_local','conc','desc_sit_loc_modif'] to merge with 2019 DataFrane in the final DataFrame . The final dataset will have an status column per year that will be used to calculate the target.

In [23]:
df_epi18_cl_d = cl.col_rest(df_epi18_cl)
df_epi17_cl_d = cl.col_rest(df_epi17_cl)
df_epi16_cl_d = cl.col_rest(df_epi16_cl)
df_epi15_cl_d = cl.col_rest(df_epi15_cl)

I check there are no duplicates and reset_index with function **cl.reset** before final merge

In [24]:
#drop duplicates
df_epi15_cl_d.drop_duplicates(inplace = True)
df_epi16_cl_d.drop_duplicates(inplace = True)
df_epi17_cl_d.drop_duplicates(inplace = True)
df_epi18_cl_d.drop_duplicates(inplace = True)
df_epi19_cl_d.drop_duplicates(inplace = True)

#id_local ordered and reset index
cl.reset(df_epi15_cl_d)
cl.reset(df_epi16_cl_d)
cl.reset(df_epi17_cl_d)
cl.reset(df_epi18_cl_d)
cl.reset(df_epi19_cl_d);

And check that it worked. There aren't nulls:

In [25]:
print('2019 Commercial premises DataFrame:')
df_epi19_cl_d.info()

2019 Commercial premises DataFrame:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 163355 entries, 0 to 163354
Data columns (total 30 columns):
id_local                   163355 non-null int64
id_distrito_local          163355 non-null int64
desc_distrito_local        163355 non-null object
id_barrio_local            163355 non-null int64
desc_barrio_local          163355 non-null object
coordenada_x_local         163355 non-null object
coordenada_y_local         163355 non-null object
desc_tipo_acceso_local     163355 non-null object
id_situacion_local         163355 non-null float64
desc_situacion_local       163355 non-null object
clase_vial_acceso          163355 non-null object
desc_vial_acceso           163355 non-null object
nom_acceso                 163355 non-null object
num_acceso                 163355 non-null int64
cal_acceso                 163355 non-null object
coordenada_x_agrupacion    163355 non-null object
coordenada_y_agrup         163355 non-null object
id_agr

In [26]:
print('2018 Commercial premises DataFrame:')
df_epi18_cl_d.info()

2018 Commercial premises DataFrame:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 146623 entries, 0 to 162466
Data columns (total 3 columns):
id_local              146623 non-null int64
conc                  146623 non-null object
desc_sit_loc_modif    146623 non-null object
dtypes: int64(1), object(2)
memory usage: 4.5+ MB


In [27]:
print('2017 Commercial premises DataFrame:')
df_epi17_cl_d.info()

2017 Commercial premises DataFrame:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 145770 entries, 0 to 161096
Data columns (total 3 columns):
id_local              145770 non-null int64
conc                  145770 non-null object
desc_sit_loc_modif    145770 non-null object
dtypes: int64(1), object(2)
memory usage: 4.4+ MB


In [28]:
print('2016 Commercial premises DataFrame:')
df_epi16_cl_d.info()

2016 Commercial premises DataFrame:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 144936 entries, 0 to 159603
Data columns (total 3 columns):
id_local              144936 non-null int64
conc                  144936 non-null object
desc_sit_loc_modif    144936 non-null object
dtypes: int64(1), object(2)
memory usage: 4.4+ MB


In [29]:
print('2015 Commercial premises DataFrame:')
df_epi15_cl_d.info()

2015 Commercial premises DataFrame:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 144057 entries, 0 to 157205
Data columns (total 3 columns):
id_local              144057 non-null int64
conc                  144057 non-null object
desc_sit_loc_modif    144057 non-null object
dtypes: int64(1), object(2)
memory usage: 4.4+ MB


I join all the info in a single Dataframe

In [30]:
columns_to_merge = ['id_local','conc']

#2015 with 2016
df_local16 = df_epi16_cl_d.merge(df_epi15_cl_d, how= 'left', 
                                    on=columns_to_merge,
                                    suffixes=('_16','_15'))
# 2016 with 2017
df_local17 = df_epi17_cl_d.merge(df_local16, how= 'left', 
                                    on=columns_to_merge,
                                    suffixes=('_17','_16'))

# 2017 with 2018
df_local18 = df_epi18_cl_d.merge(df_local17, how= 'left', 
                                    on=columns_to_merge,
                                    suffixes=('_18','_17'))

# 2018 with 2019
df_local19 = df_epi19_cl_d.merge(df_local18, how= 'left', 
                                    on=columns_to_merge,
                                    suffixes=('_19','_18'))
names = df_local19.columns.tolist()
names[names.index('desc_sit_loc_modif')] = 'desc_sit_loc_modif_19'
df_local19.columns = names

print('Commercial premises merged file shape:', df_local19.shape)
print('Commercial premises merged file inique id_local: %0.0f' %(len(df_local19['id_local'].unique())))
print('Commercial premises merged file columns:', df_local19.columns.values)
print('Commercial premises merged file columns info:') 
df_local19.info()

Commercial premises merged file shape: (163652, 34)
Commercial premises merged file inique id_local: 147344
Commercial premises merged file columns: ['id_local' 'id_distrito_local' 'desc_distrito_local' 'id_barrio_local'
 'desc_barrio_local' 'coordenada_x_local' 'coordenada_y_local'
 'desc_tipo_acceso_local' 'id_situacion_local' 'desc_situacion_local'
 'clase_vial_acceso' 'desc_vial_acceso' 'nom_acceso' 'num_acceso'
 'cal_acceso' 'coordenada_x_agrupacion' 'coordenada_y_agrup'
 'id_agrupacion' 'nombre_agrupacion' 'id_tipo_agrup' 'desc_tipo_agrup'
 'rotulo' 'id_seccion' 'desc_seccion' 'id_division' 'desc_division'
 'id_epigrafe' 'desc_epigrafe' 'conc' 'desc_sit_loc_modif_19'
 'desc_sit_loc_modif_18' 'desc_sit_loc_modif_17' 'desc_sit_loc_modif_16'
 'desc_sit_loc_modif_15']
Commercial premises merged file columns info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 163652 entries, 0 to 163651
Data columns (total 34 columns):
id_local                   163652 non-null int64
id_distrito_l

Some numeric columns have 'object' type ('id_epigrafe', 'id_division'). I drop non numeric values because they are of no use for the analysis (PTECO1 == 'activity pending to code') and convert to numeric

In [31]:
df_local19_f = df_local19[(df_local19.id_epigrafe != 'PTECO1')].copy()
cl.reset(df_local19_f);
cols = ['id_division','id_epigrafe']
df_local19_f.loc[:,cols] = df_local19_f.loc[:,cols].applymap(np.int64);
df_local19_f.id_division.values
print('Commercial premises merged file columns dtypes:') 
df_local19_f.dtypes

Commercial premises merged file columns dtypes:


id_local                     int64
id_distrito_local            int64
desc_distrito_local         object
id_barrio_local              int64
desc_barrio_local           object
coordenada_x_local          object
coordenada_y_local          object
desc_tipo_acceso_local      object
id_situacion_local         float64
desc_situacion_local        object
clase_vial_acceso           object
desc_vial_acceso            object
nom_acceso                  object
num_acceso                   int64
cal_acceso                  object
coordenada_x_agrupacion     object
coordenada_y_agrup          object
id_agrupacion              float64
nombre_agrupacion           object
id_tipo_agrup              float64
desc_tipo_agrup             object
rotulo                      object
id_seccion                  object
desc_seccion                object
id_division                  int64
desc_division               object
id_epigrafe                  int64
desc_epigrafe               object
conc                

### 4) Generate target variable

First of all, with **cl.sit_year** I generate variables open and closed for each each year: I compare the situation of every local with the previous years ('Cerrado' for closed and 'NaN' for open).   
Example:

     cond1 = ((df_.desc_sit_loc_modif_19 == 'Cerrado') |
                                 (df_.desc_sit_loc_modif_19 == 'Uso vivienda'))     
                                 
     cond2 = ((df_.desc_sit_loc_modif_18 != 'Cerrado') |
                                 df.desc_sit_loc_modif_18.isnull())   
                                 
     cond3 = ((df_.desc_sit_loc_modif_19.notnull()) &
                                 (df_.desc_sit_loc_modif_18.isnull()))   
 
    df_['cerrado_19'] = np.where(cond1 & cond2, 1 ,0)   
    df_['abierto_19'] = np.where(cond3, 1 ,0)

In [32]:
df_local19_f = cl.sit_year(df_local19_f)
print('New columns opened and closed per year:', df_local19.columns.values)

New columns opened and closed per year: ['id_local' 'id_distrito_local' 'desc_distrito_local' 'id_barrio_local'
 'desc_barrio_local' 'coordenada_x_local' 'coordenada_y_local'
 'desc_tipo_acceso_local' 'id_situacion_local' 'desc_situacion_local'
 'clase_vial_acceso' 'desc_vial_acceso' 'nom_acceso' 'num_acceso'
 'cal_acceso' 'coordenada_x_agrupacion' 'coordenada_y_agrup'
 'id_agrupacion' 'nombre_agrupacion' 'id_tipo_agrup' 'desc_tipo_agrup'
 'rotulo' 'id_seccion' 'desc_seccion' 'id_division' 'desc_division'
 'id_epigrafe' 'desc_epigrafe' 'conc' 'desc_sit_loc_modif_19'
 'desc_sit_loc_modif_18' 'desc_sit_loc_modif_17' 'desc_sit_loc_modif_16'
 'desc_sit_loc_modif_15']


Second: I generated a target variable (target1): locals opened and closed in less that 2 years since 2016 but I don't have enough '1s':
- less than 0,3% including premises with desc_epigrafe == 'LOCAL SIN ACTIVIDAD'
- less than 0,3% without premises with desc_epigrafe == 'LOCAL SIN ACTIVIDAD'

In [33]:
# I generate a target variable: locals opened and closed in less that 2 years since 2016 but I don't have enough '1s'
df_local19_ft2 = cl.target(df_local19_f,2,2016)
df_local19_ft2.head()

print('Target1/total (with "No activity" premises): % .4f' %(len(df_local19_ft2[df_local19_ft2.target == 1])/len(df_local19_ft2))) 
print('Target1/total (without "No activity" premises): % .4f' %(len(df_local19_ft2[(df_local19_ft2.target == 1) 
                                                                              & (df_local19_ft2.desc_epigrafe != 'LOCAL SIN ACTIVIDAD')])/
                                                           len(df_local19_ft2[df_local19_ft2.desc_epigrafe != 'LOCAL SIN ACTIVIDAD'])))

Target1/total (with "No activity" premises):  0.0033
Target1/total (without "No activity" premises):  0.0030


So, I generated an new target variable (target2): activities or premises closed since 2017 (3 years timeframe)  
I have still a very umbalanced dataset but at least over 2% target2 vs global.   
Target 2 will be the target that I will manage in the modelling phase and from now onwards, I will call it 'target' variable.

In [34]:
df_local19_ft = cl.target2(df_local19_f,3)
df_local19_ft.head()

print('Target2/total (with "No activity" premises): % .4f' %(len(df_local19_ft[df_local19_ft.target == 1])/len(df_local19_ft))) 
print('Target2/total (without "No activity" premises): % .4f' %(len(df_local19_ft[(df_local19_ft.target == 1) 
                                                                              & (df_local19_ft.desc_epigrafe != 'LOCAL SIN ACTIVIDAD')])/
                                                           len(df_local19_ft[df_local19_ft.desc_epigrafe != 'LOCAL SIN ACTIVIDAD'])))


Target2/total (with "No activity" premises):  0.0673
Target2/total (without "No activity" premises):  0.0233


### 5) Standardize types of activities

Now I standardize the type of activity: count of desc_epigrafe (activity) less than percentile .75 are grouped in a common activity == 'OTRAS ACTIVIDADES'.    
The new variables generated are:   
- desc_act_norm: desc_epigrafe == 'OTRAS ACTIVIDADES' for activities below percentile .75(#locals/activity) == 254
- id_act_norm: id_epigrafe == 999999 for activities below percentile .75(#locals/activity) == 254

In [35]:
print('Type of activities quantile distribution:')
df_local19_ft.id_epigrafe.value_counts().quantile([.1, .25, .5, .75]) 

Type of activities quantile distribution:


0.10      5.0
0.25     19.0
0.50     82.5
0.75    254.0
Name: id_epigrafe, dtype: float64

In [36]:
# Standardize activities
act = df_local19_ft.desc_epigrafe.value_counts()[df_local19_ft.desc_epigrafe.value_counts() < df_local19_ft.id_epigrafe.value_counts().quantile([.75]).values[0]].index
df_local19_ft['desc_act_norm'] = df_local19_ft.apply(lambda x: cl.norm_act(x['desc_epigrafe'],act),axis=1)
df_local19_ft.loc[:,'id_act_norm'] = df_local19_ft.apply(lambda x: cl.norm_id_act(x['desc_epigrafe'],x['id_epigrafe'],act),axis=1)

# Check results
df_oa = df_local19_ft[df_local19_ft.desc_act_norm == 'OTRAS ACTIVIDADES']
print('Number of premises grouped in "OTRAS ACTIVIDADES" % 0.0f' % len(df_oa))
print('#1s in target "OTRAS ACTIVIDADES" / total "OTRAS ACTIVIDADES" % 0.2f' % (len(df_oa[df_oa.target == 1])/len(df_oa)))
print('Value counts / activity:')
pr = pd.DataFrame(df_local19_ft.desc_act_norm.value_counts())
pr

Number of premises grouped in "OTRAS ACTIVIDADES"  21503
#1s in target "OTRAS ACTIVIDADES" / total "OTRAS ACTIVIDADES"  0.02
Value counts / activity:


,desc_act_norm
LOCAL SIN ACTIVIDAD,42782
OTRAS ACTIVIDADES,21503
SERVICIO DE PELUQUERIA,5260
BAR CON COCINA,4572
COMERCIO AL POR MENOR DE PRENDAS DE VESTIR EN ESTABLECIMIENTOS ESPECIALIZADOS,4470
BAR RESTAURANTE,4320
CAFETERIA,3247
RESTAURANTE,2656
OTRO COMERCIO AL POR MENOR DE PRODUCTOS ALIMENTICIOS (PERECEDEROS Y NO PERECEDEROS) CON VENDEDOR N.C.O.P.,2617
COMERCIO AL POR MENOR DE FRUTAS Y HORTALIZAS SIN OBRADOR,2456


### 6) Generate interesting variables and convert UTM coordinates

First, I generate interesting variables (KPIs) with the script **cl.kpis_total**:   
- 'ab_17_19': '1' if the local has been opened in the last 3 years and '0' on the contrary
- total activity/district, total 'ab_17_19'/activity/district, total activity/neighbourhood, total 'ab_17_19'/activity/neighbourhood   
- 'no activity' vs total ratio per distric and neighbourghood   

and I check that the new variables have been created correctly

In [37]:
df_local19_ft = cl.kpis_total(df_local19_ft)
df_locals = df_local19_ft.copy()
cl.reset(df_locals)
print('New columns of KPIs:', df_locals.columns.values)
df_locals.head(2)

New columns of KPIs: ['id_local' 'id_distrito_local' 'desc_distrito_local' 'id_barrio_local'
 'desc_barrio_local' 'coordenada_x_local' 'coordenada_y_local'
 'desc_tipo_acceso_local' 'id_situacion_local' 'desc_situacion_local'
 'clase_vial_acceso' 'desc_vial_acceso' 'nom_acceso' 'num_acceso'
 'cal_acceso' 'coordenada_x_agrupacion' 'coordenada_y_agrup'
 'id_agrupacion' 'nombre_agrupacion' 'id_tipo_agrup' 'desc_tipo_agrup'
 'rotulo' 'id_seccion' 'desc_seccion' 'id_division' 'desc_division'
 'id_epigrafe' 'desc_epigrafe' 'conc' 'desc_sit_loc_modif_19'
 'desc_sit_loc_modif_18' 'desc_sit_loc_modif_17' 'desc_sit_loc_modif_16'
 'desc_sit_loc_modif_15' 'cerrado_19' 'abierto_19' 'cerrado_18'
 'abierto_18' 'cerrado_17' 'abierto_17' 'cerrado_16' 'abierto_16' 'target'
 'desc_act_norm' 'id_act_norm' 'ab_17_19' 'loc_dist_act'
 'ab_dist_act_17_19' 'total_loc_act' 'total_ab_act_17_19' 'loc_dist'
 'ab_dist_17_19' 'loc_na_dist' 'ab_dist_act_17_19_rate'
 'total_ab_act_17_19_rate' 'total_ab_dist_17_19_rate

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,coordenada_x_local,coordenada_y_local,desc_tipo_acceso_local,id_situacion_local,desc_situacion_local,clase_vial_acceso,desc_vial_acceso,nom_acceso,num_acceso,cal_acceso,coordenada_x_agrupacion,coordenada_y_agrup,id_agrupacion,nombre_agrupacion,id_tipo_agrup,desc_tipo_agrup,rotulo,id_seccion,desc_seccion,id_division,desc_division,id_epigrafe,desc_epigrafe,conc,desc_sit_loc_modif_19,desc_sit_loc_modif_18,desc_sit_loc_modif_17,desc_sit_loc_modif_16,desc_sit_loc_modif_15,cerrado_19,abierto_19,cerrado_18,abierto_18,cerrado_17,abierto_17,cerrado_16,abierto_16,target,desc_act_norm,id_act_norm,ab_17_19,loc_dist_act,ab_dist_act_17_19,total_loc_act,total_ab_act_17_19,loc_dist,ab_dist_17_19,loc_na_dist,ab_dist_act_17_19_rate,total_ab_act_17_19_rate,total_ab_dist_17_19_rate,total_na_dist_rate,loc_barrio_act,ab_barrio_act_17_19,loc_barrio,ab_barrio_17_19,loc_na_barrio,ab_barrio_act_17_19_rate,total_ab_barr_17_19_rate,total_na_barr_rate
0,270440895,3,RETIRO,304,IBIZA,0,0,Agrupado,1.0,Abierto,CALLE,IBIZA ...,NUM,8,,"442533,58","4474389,51",99000191.0,MERCADO MUNICIPAL DE IBIZA,12.0,Mercado Municipal,BAR HERMANOS BENAYAS,I,HOSTELERIA,56,SERVICIOS DE COMIDAS Y BEBIDAS,563005,BAR SIN COCINA,BAR HERMANOS BENAYAS-IBIZA-8,Abierto,Abierto,Abierto,Abierto,Abierto,0,0,0,0,0,0,0,0,0,BAR SIN COCINA,563005,0,71,23.0,1700,428.0,4609,900,559,32.394366,25.176471,19.527012,12.128444,17,6.0,1066,247,54,35.294118,23.170732,5.065666
1,270440899,5,CHAMARTIN,502,PROSPERIDAD,"443460,59","4477604,5",Puerta Calle,1.0,Abierto,CALLE,CORAZON DE MARIA ...,NUM,57,,0,0,-1.0,SIN AGRUPACION,-1.0,SIN AGRUPACION,LA PARRILLA BERENGUER,I,HOSTELERIA,56,SERVICIOS DE COMIDAS Y BEBIDAS,561005,BAR CON COCINA,LA PARRILLA BERENGUER-CORAZON DE MARIA-57,Abierto,Abierto,NaN,NaN,NaN,0,0,0,1,0,0,0,0,0,BAR CON COCINA,561005,1,176,31.0,4572,867.0,7652,1254,1301,17.613636,18.963255,16.387872,17.002091,49,12.0,1841,330,356,24.489796,17.925041,19.337317


Second, I generate another variable **'num_act'** with the number of diferent activities licenced for a single local 

In [38]:
df_locals_v1 = pd.DataFrame(df_locals, columns=['id_local','desc_epigrafe'])
df_locals_v1['act'] = int(1)
df_locals_v1.drop_duplicates(inplace = True)

df_locals_v2 = df_locals_v1.pivot(index='id_local', columns='desc_epigrafe', values='act')
df_locals_v2.fillna(0, inplace=True)
df_locals_v3 = df_locals_v2.astype(int).reset_index()
df_locals_v3['num_act'] = df_locals_v3.sum(axis =1).astype(np.int64) - df_locals_v3['id_local']
df_locals_v3 = df_locals_v3[['id_local','num_act']]
print('Check that the variable "num_act" is created (first 5 lines):')
df_locals_v3.head()

Check that the variable "num_act" is created (first 5 lines):


desc_epigrafe,id_local,num_act
0,10000003,2
1,10000004,1
2,10000013,1
3,10000044,1
4,10000052,1


Now:   
- I merge with locals file and generate the global DataFrame dropping duplicated id_locals

In [39]:
# compares unique locals with those with multiple activities
df_locals_v4 = df_locals.merge(df_locals_v3, how='left',on=['id_local']).drop_duplicates(subset='id_local', keep='first')
cl.reset(df_locals_v4)
print('Total rows in the DataFrame: %0.0f' % (len(df_locals_v4)))
print('Total unique premises: %0.0f' % (len(df_locals_v4.id_local.unique())))
print('Total premises with more that one activity licenced: %0.0f' % (len(df_locals_v4[df_locals_v4.num_act > 1])))

Total rows in the DataFrame: 147334
Total unique premises: 147334
Total premises with more that one activity licenced: 11754


- I convert UTM into geographical coordinates and count the number of locals of the same activity within a range (this function requires a lot of compute capacity, I will apply it only for analysis of one local each time). 

Note: First I have to unify colective groups coordinate with single commerce coordinates in the same variable

In [40]:
df_locals_v5 = df_locals_v4.copy()
df_locals_v5['coord_x_f'] = [c.replace(',', '.') for c in df_locals_v5['coordenada_x_local'].values]
df_locals_v5['coord_y_f'] = [c.replace(',', '.') for c in df_locals_v5['coordenada_y_local'].values]
df_locals_v5['coord_x_af'] = [c.replace(',', '.') for c in df_locals_v5['coordenada_x_agrupacion'].astype(str).values]
df_locals_v5['coord_y_af'] = [c.replace(',', '.') for c in df_locals_v5['coordenada_y_agrup'].astype(str).values]

df_locals_v5['coord_x_f'] = df_locals_v5['coord_x_f'].map(float)
df_locals_v5['coord_y_f'] = df_locals_v5['coord_y_f'].map(float)
df_locals_v5['coord_x_af'] = df_locals_v5['coord_x_af'].map(float)
df_locals_v5['coord_y_af'] = df_locals_v5['coord_y_af'].map(float)

df_locals_v5['coord_x_final'] = np.where(df_locals_v5['desc_tipo_acceso_local'] == 'Agrupado', df_locals_v5['coord_x_af'] , df_locals_v5['coord_x_f'])
df_locals_v5['coord_y_final'] = np.where(df_locals_v5['desc_tipo_acceso_local'] == 'Agrupado', df_locals_v5['coord_y_af'] , df_locals_v5['coord_y_f'])

I generate lat and lon coordinates with utm.tolatlon function. Assumptions:   
- Madrid is in zone 30 T   
- I have to drop coor_x == 0 because can't not be converted into geographic coordinates because it does't exist. They are less than 3% of the samples so I assume the loss

In [41]:
import utm

In [42]:
print('perc of samples/total with coord_x = "0": %0.2f' % (len(df_locals_v5[df_locals_v5['coord_x_final'] == 0])/len(df_locals_v5)))
df_locals_v6 = df_locals_v5[df_locals_v5.coord_x_final != 0].copy()
cl.reset(df_locals_v6);
df_locals_v6.loc[df_locals_v6.index.values,'lat'],df_locals_v6.loc[df_locals_v6.index.values,'lon'] = utm.to_latlon(df_locals_v6.loc[df_locals_v6.index.values,'coord_x_final'],df_locals_v6.loc[df_locals_v6.index.values,'coord_y_final'],30,'T')

perc of samples/total with coord_x = "0": 0.03


I check shape, columns and head of the DataFrame to see how the look like and be sure that the new coordinates have been created correctly:

In [43]:
df_locals_v6.shape

(143191, 74)

In [44]:
df_locals_v6.columns.values

array(['id_local', 'id_distrito_local', 'desc_distrito_local',
       'id_barrio_local', 'desc_barrio_local', 'coordenada_x_local',
       'coordenada_y_local', 'desc_tipo_acceso_local',
       'id_situacion_local', 'desc_situacion_local', 'clase_vial_acceso',
       'desc_vial_acceso', 'nom_acceso', 'num_acceso', 'cal_acceso',
       'coordenada_x_agrupacion', 'coordenada_y_agrup', 'id_agrupacion',
       'nombre_agrupacion', 'id_tipo_agrup', 'desc_tipo_agrup', 'rotulo',
       'id_seccion', 'desc_seccion', 'id_division', 'desc_division',
       'id_epigrafe', 'desc_epigrafe', 'conc', 'desc_sit_loc_modif_19',
       'desc_sit_loc_modif_18', 'desc_sit_loc_modif_17',
       'desc_sit_loc_modif_16', 'desc_sit_loc_modif_15', 'cerrado_19',
       'abierto_19', 'cerrado_18', 'abierto_18', 'cerrado_17',
       'abierto_17', 'cerrado_16', 'abierto_16', 'target',
       'desc_act_norm', 'id_act_norm', 'ab_17_19', 'loc_dist_act',
       'ab_dist_act_17_19', 'total_loc_act', 'total_ab_act_17_19'

In [45]:
df_locals_v6.head(2)

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,coordenada_x_local,coordenada_y_local,desc_tipo_acceso_local,id_situacion_local,desc_situacion_local,clase_vial_acceso,desc_vial_acceso,nom_acceso,num_acceso,cal_acceso,coordenada_x_agrupacion,coordenada_y_agrup,id_agrupacion,nombre_agrupacion,id_tipo_agrup,desc_tipo_agrup,rotulo,id_seccion,desc_seccion,id_division,desc_division,id_epigrafe,desc_epigrafe,conc,desc_sit_loc_modif_19,desc_sit_loc_modif_18,desc_sit_loc_modif_17,desc_sit_loc_modif_16,desc_sit_loc_modif_15,cerrado_19,abierto_19,cerrado_18,abierto_18,cerrado_17,abierto_17,cerrado_16,abierto_16,target,desc_act_norm,id_act_norm,ab_17_19,loc_dist_act,ab_dist_act_17_19,total_loc_act,total_ab_act_17_19,loc_dist,ab_dist_17_19,loc_na_dist,ab_dist_act_17_19_rate,total_ab_act_17_19_rate,total_ab_dist_17_19_rate,total_na_dist_rate,loc_barrio_act,ab_barrio_act_17_19,loc_barrio,ab_barrio_17_19,loc_na_barrio,ab_barrio_act_17_19_rate,total_ab_barr_17_19_rate,total_na_barr_rate,num_act,coord_x_f,coord_y_f,coord_x_af,coord_y_af,coord_x_final,coord_y_final,lat,lon
0,270440895,3,RETIRO,304,IBIZA,0,0,Agrupado,1.0,Abierto,CALLE,IBIZA ...,NUM,8,,"442533,58","4474389,51",99000191.0,MERCADO MUNICIPAL DE IBIZA,12.0,Mercado Municipal,BAR HERMANOS BENAYAS,I,HOSTELERIA,56,SERVICIOS DE COMIDAS Y BEBIDAS,563005,BAR SIN COCINA,BAR HERMANOS BENAYAS-IBIZA-8,Abierto,Abierto,Abierto,Abierto,Abierto,0,0,0,0,0,0,0,0,0,BAR SIN COCINA,563005,0,71,23.0,1700,428.0,4609,900,559,32.394366,25.176471,19.527012,12.128444,17,6.0,1066,247,54,35.294118,23.170732,5.065666,1,0.00,0.0,442533.58,4474389.51,442533.58,4474389.51,40.418148,-3.677374
1,270440899,5,CHAMARTIN,502,PROSPERIDAD,"443460,59","4477604,5",Puerta Calle,1.0,Abierto,CALLE,CORAZON DE MARIA ...,NUM,57,,0,0,-1.0,SIN AGRUPACION,-1.0,SIN AGRUPACION,LA PARRILLA BERENGUER,I,HOSTELERIA,56,SERVICIOS DE COMIDAS Y BEBIDAS,561005,BAR CON COCINA,LA PARRILLA BERENGUER-CORAZON DE MARIA-57,Abierto,Abierto,NaN,NaN,NaN,0,0,0,1,0,0,0,0,0,BAR CON COCINA,561005,1,176,31.0,4572,867.0,7652,1254,1301,17.613636,18.963255,16.387872,17.002091,49,12.0,1841,330,356,24.489796,17.925041,19.337317,1,443460.59,4477604.5,0.00,0.00,443460.59,4477604.50,40.447174,-3.666734


### 7) Add population info

I combine the population info: census and floating population and check the features:

In [46]:
censo = df1.merge(df2,how='left', on='id_distrito_local')
censo = censo.drop(columns=['distrito','Una mujer sola de 16 a 64 años ', 'Un hombre solo de 16 a 64 años',
       'Una mujer sola de 65 o más años', 'Un hombre solo de 65 o más años',
       'Una mujer adulta  con uno o más menores',
       'Un hombre adulto  con uno o más menores',
       'Dos adultos de 16 a 64 años, sin menores',
       'Dos adultos, uno al menos de 65 o más años, sin menores',
       'Dos adultos y un menor', 'Dos adultos y dos menores',
       'Dos adultos y tres o más menores',
       'Dos adultos de 35 años o más, otro de 16 a 34 años, sin menores',
       'Dos adultos de 35 años o más, otro de 16 a 34 años y un menor',
       'Dos adultos de 35 años o más, otro de 16 a 34 años y dos o más menores',
       'Otro hogar de tres adultos, con o sin menores',
       'Dos adultos de 35 años o más, dos de 16 a 34 años, sin menores',
       'Dos adultos de 35 años o más, dos de 16 a 34 años y un menor',
       'Dos adultos de 35 años o más, dos  de 16 a 34 años y dos o más menores',
       'Otro hogar de cuatro adultos, con o sin menores',
       'Cinco o más adultos, con o sin menores',
       'Hogar con 15 ó más habitantes', 'Hogares con menores solos'])
censo.columns

Index(['id_distrito_local', 'Población', 'Hombre', 'Mujeres',
       'Densidad (Habitantes / Ha.)', 'Edad promedio', 'Edad mediana',
       'Renta media/persona (euros)', 'Total Hogares', 'Españoles',
       'Extranjeros', 'Mixtos', 'España fuera barrio dia laboral',
       'Extranjero fuera barrio dia laboral', 'Total fuera barrio dia laboral',
       'España fuera barrio fin semana', 'Extranjero fuera barrio fin semana',
       'Total fuera barrio fin semana', 'Total barrio dia laboral',
       'Total barrio fin semana', 'Total trabajo dia laboral',
       'Total trabajo fin semana'],
      dtype='object')

I calculate new variables: working, residential and out of neighbourhood vs total week floating population ratio

In [47]:
censo['total_TF_week']= (censo['Total trabajo fin semana']
                         + censo['Total fuera barrio fin semana']
                         + censo['Total barrio fin semana']
                         + censo['Total trabajo dia laboral']
                         + censo['Total fuera barrio dia laboral']
                         + censo['Total barrio dia laboral'])

censo['ratio_t_total'] = (censo['Total trabajo fin semana']
                          +censo['Total trabajo dia laboral'])/censo['total_TF_week']
censo['ratio_fb_total'] = (censo['Total fuera barrio fin semana']
                           +censo['Total fuera barrio dia laboral'])/censo['total_TF_week']
censo['ratio_b_total'] = (censo['Total barrio fin semana']
                          +censo['Total barrio dia laboral'])/censo['total_TF_week']

I create a new variable 'dist_type' to codify the type of district:
- 'Commercial' == 1
- 'Residential' == 2
- 'Mix' == 3 

In [48]:
fb_75 = np.percentile(censo['ratio_fb_total'], 75)
b_75 = np.percentile(censo['ratio_b_total'], 75)

def dist_type(ratio_fb,ratio_b):
    if ratio_fb > fb_75:
        return 1
    elif ratio_b > b_75:
        return 2
    else:
        return 3
censo['dist_type'] = censo.apply(lambda x: dist_type(x['ratio_fb_total'],x['ratio_b_total']),axis=1)

I check how the population census information looks like

In [49]:
censo.head()

,id_distrito_local,Población,Hombre,Mujeres,Densidad (Habitantes / Ha.),Edad promedio,Edad mediana,Renta media/persona (euros),Total Hogares,Españoles,Extranjeros,Mixtos,España fuera barrio dia laboral,Extranjero fuera barrio dia laboral,Total fuera barrio dia laboral,España fuera barrio fin semana,Extranjero fuera barrio fin semana,Total fuera barrio fin semana,Total barrio dia laboral,Total barrio fin semana,Total trabajo dia laboral,Total trabajo fin semana,total_TF_week,ratio_t_total,ratio_fb_total,ratio_b_total,dist_type
0,1,115198,58027,57171,257.985172,44.026383,42.109404,16147,67296,50164,8979,8153,1956512,348482,2304994,960027,169516,1129543,623007,241830,363775,110026,4773175,0.099263,0.719550,0.181187,1
1,2,123013,56064,66949,238.046748,44.545401,44.311168,17306,65005,56759,2798,5448,900033,45873,945906,359245,18421,377666,655916,254049,190696,55740,2479973,0.099370,0.533704,0.366925,3
2,3,95784,42275,53509,218.394417,47.041636,47.494624,21504,48673,43211,1881,3581,1085546,115080,1200626,423688,52559,476247,512972,197762,222149,64114,2673870,0.107059,0.627133,0.265807,1
3,4,117988,50420,67568,271.025704,46.241563,45.492514,24433,62951,51490,5405,6056,1299023,122121,1421144,517838,55491,573329,625927,241789,377950,111528,3351667,0.146040,0.595069,0.258891,3
4,5,114571,49918,64653,158.972579,45.388980,45.175239,25969,58240,49825,3021,5394,1482648,112338,1594986,530814,41799,572613,629355,242013,389699,106530,3535196,0.140368,0.613148,0.246484,1


And the info available have homogeneous types and there aren't nulls:

In [50]:
censo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 0 to 20
Data columns (total 27 columns):
id_distrito_local                      21 non-null int64
Población                              21 non-null int64
Hombre                                 21 non-null int64
Mujeres                                21 non-null int64
Densidad (Habitantes / Ha.)            21 non-null float64
Edad promedio                          21 non-null float64
Edad mediana                           21 non-null float64
Renta media/persona (euros)            21 non-null int64
Total Hogares                          21 non-null int64
Españoles                              21 non-null int64
Extranjeros                            21 non-null int64
Mixtos                                 21 non-null int64
España fuera barrio dia laboral        21 non-null int64
Extranjero fuera barrio dia laboral    21 non-null int64
Total fuera barrio dia laboral         21 non-null int64
España fuera barrio fin semana      

I check the statistics. The features with a more spread range are those related to floating traffic within districts.   
Anyhow, since all the values are agreggated per district, the outliers are intrinsic characteristics to the district.

In [51]:
censo.describe()

,id_distrito_local,Población,Hombre,Mujeres,Densidad (Habitantes / Ha.),Edad promedio,Edad mediana,Renta media/persona (euros),Total Hogares,Españoles,Extranjeros,Mixtos,España fuera barrio dia laboral,Extranjero fuera barrio dia laboral,Total fuera barrio dia laboral,España fuera barrio fin semana,Extranjero fuera barrio fin semana,Total fuera barrio fin semana,Total barrio dia laboral,Total barrio fin semana,Total trabajo dia laboral,Total trabajo fin semana,total_TF_week,ratio_t_total,ratio_fb_total,ratio_b_total,dist_type
count,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,2.100000e+01,21.000000,2.100000e+01,21.000000,21.000000,2.100000e+01,2.100000e+01,21.000000,21.000000,21.000000,2.100000e+01,21.000000,21.000000,21.000000,21.000000
mean,11.000000,120238.523810,54560.952381,65677.571429,142.549481,43.907969,44.060552,16061.285714,61436.380952,51351.142857,3957.523810,6127.714286,9.139553e+05,69671.047619,9.836263e+05,363881.047619,29698.857143,3.935799e+05,6.680512e+05,259142.714286,234471.761905,66906.761905,2.605779e+06,0.113124,0.514946,0.371930,2.285714
std,6.204837,43415.307389,19672.393300,23920.862130,98.207354,2.243709,2.103119,5049.098337,21613.443271,17825.203798,2117.751228,2728.174191,3.967712e+05,77584.497180,4.596726e+05,181078.054614,37716.764627,2.139075e+05,2.354600e+05,91519.726899,100183.333120,27777.790499,8.601518e+05,0.021090,0.108750,0.120509,0.845154
min,1.000000,35949.000000,16869.000000,19080.000000,10.344042,38.825609,39.567590,9395.000000,18269.000000,15912.000000,815.000000,1542.000000,3.065860e+05,4929.000000,3.115150e+05,122010.000000,2035.000000,1.240450e+05,2.020600e+05,78679.000000,80461.000000,24059.000000,1.124201e+06,0.083173,0.356969,0.172583,1.000000
25%,6.000000,95784.000000,42275.000000,53509.000000,68.661063,42.279377,42.672722,11925.000000,48673.000000,39013.000000,2346.000000,4341.000000,6.301550e+05,17628.000000,6.476170e+05,246339.000000,7104.000000,2.526670e+05,5.337300e+05,205283.000000,153482.000000,42229.000000,1.901033e+06,0.092973,0.430957,0.264063,2.000000
50%,11.000000,115198.000000,51143.000000,64653.000000,156.841593,44.026383,43.395233,15048.000000,62013.000000,51347.000000,3361.000000,5817.000000,8.655940e+05,53443.000000,8.995260e+05,326400.000000,19866.000000,3.439330e+05,6.293550e+05,242013.000000,222149.000000,64114.000000,2.643653e+06,0.114327,0.499644,0.377971,3.000000
75%,16.000000,139147.000000,64183.000000,74964.000000,218.394417,45.783141,45.175239,18573.000000,70099.000000,60925.000000,5405.000000,8153.000000,1.156001e+06,86168.000000,1.216536e+06,423688.000000,39454.000000,4.762470e+05,7.651690e+05,293485.000000,317615.000000,90215.000000,3.152694e+06,0.129359,0.606578,0.473789,3.000000
max,21.000000,192933.000000,87414.000000,105519.000000,298.017711,47.545059,48.871349,25969.000000,95680.000000,80186.000000,8979.000000,11865.000000,1.956512e+06,348482.000000,2.304994e+06,960027.000000,169516.000000,1.129543e+06,1.093869e+06,427359.000000,389699.000000,111528.000000,4.773175e+06,0.146505,0.719550,0.553191,3.000000


I merge locals with population census and clean pivot columns (ie:coordinates). Now my base dataset contains: 143191 id_locals with 92 features

In [52]:
df_locals_v7 = df_locals_v6.merge(censo, how='left', on='id_distrito_local').sort_values('id_local').reset_index(drop=True)
df_locals_v7.fillna('No disponible',inplace=True)
df_locals_v7.drop(columns=['coordenada_x_local','coordenada_y_local',
                           'coordenada_x_agrupacion','coordenada_y_agrup',
                           'coord_x_f','coord_y_f', 'coord_x_af','coord_y_af'],inplace=True)
cl.reset(df_locals_v7)
df_locals_v7.columns, df_locals_v7.shape

(Index(['id_local', 'id_distrito_local', 'desc_distrito_local',
        'id_barrio_local', 'desc_barrio_local', 'desc_tipo_acceso_local',
        'id_situacion_local', 'desc_situacion_local', 'clase_vial_acceso',
        'desc_vial_acceso', 'nom_acceso', 'num_acceso', 'cal_acceso',
        'id_agrupacion', 'nombre_agrupacion', 'id_tipo_agrup',
        'desc_tipo_agrup', 'rotulo', 'id_seccion', 'desc_seccion',
        'id_division', 'desc_division', 'id_epigrafe', 'desc_epigrafe', 'conc',
        'desc_sit_loc_modif_19', 'desc_sit_loc_modif_18',
        'desc_sit_loc_modif_17', 'desc_sit_loc_modif_16',
        'desc_sit_loc_modif_15', 'cerrado_19', 'abierto_19', 'cerrado_18',
        'abierto_18', 'cerrado_17', 'abierto_17', 'cerrado_16', 'abierto_16',
        'target', 'desc_act_norm', 'id_act_norm', 'ab_17_19', 'loc_dist_act',
        'ab_dist_act_17_19', 'total_loc_act', 'total_ab_act_17_19', 'loc_dist',
        'ab_dist_17_19', 'loc_na_dist', 'ab_dist_act_17_19_rate',
        'total

And the info available have homogeneous types and there aren't nulls:

In [53]:
df_locals_v7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143191 entries, 0 to 143190
Data columns (total 92 columns):
id_local                               143191 non-null int64
id_distrito_local                      143191 non-null int64
desc_distrito_local                    143191 non-null object
id_barrio_local                        143191 non-null int64
desc_barrio_local                      143191 non-null object
desc_tipo_acceso_local                 143191 non-null object
id_situacion_local                     143191 non-null float64
desc_situacion_local                   143191 non-null object
clase_vial_acceso                      143191 non-null object
desc_vial_acceso                       143191 non-null object
nom_acceso                             143191 non-null object
num_acceso                             143191 non-null int64
cal_acceso                             143191 non-null object
id_agrupacion                          143191 non-null float64
nombre_agrupacion      

### 8) Select activities for analysis

I will filter the types of activities with the lower target rate/total ratio to focus on premises with higher rate of closures

I will remove premises with activity = 'LOCAL SIN ACTIVIDAD' since one of the objectives of the study is to identify type of commercial premises with more propensity to close and 'No activity' premises add no information.  

After removing premises with 'No activity', the target/total ratio drops till 2%.

I filter the activities with the lowest target/total ratio to increase the number of '1s' vs total with the function **cl.act_filter_id_norm**.

As a result, we see the activities with more 4% of closed premises between 2017 and 2019. I will use this DataFrame as a base for the first iteration of my classification models

In [54]:
df_locals_v8 = df_locals_v7[df_locals_v7.id_epigrafe != 0]
print('Target/total rate after dropping "LOCAL NO ACTIVIDAD": %0.2f' % (len(df_locals_v8[df_locals_v8.target == 1])/len(df_locals_v8)))
id_act_fil = cl.act_filter_id_norm(df_locals_v8,0.035)
desc_act_fil = cl.act_filter_desc_norm(df_locals_v8,0.035)
for i in desc_act_fil.index.values:
       desc_act_fil.loc[desc_act_fil.index == i,'id'] = df_locals_v8.loc[df_locals_v8.desc_epigrafe == i].id_epigrafe.unique()[0]
desc_act_fil['id'] = desc_act_fil['id'].apply(np.int)
print('Activities with a target/total rate > 0.035:')
desc_act_fil

Target/total rate after dropping "LOCAL NO ACTIVIDAD": 0.02
Activities with a target/total rate > 0.035:


,act_count,ones,perc,id
CENTROS DE BRONCEADO,84,8.0,0.095238,960205
COMERCIO AL POR MENOR DE PESCADOS Y MARISCOS SIN OBRADOR,667,56.0,0.083958,472302
COMERCIO AL POR MENOR DE CHARCUTERIA,557,41.0,0.073609,472202
COMERCIO AL POR MENOR DE ARTICULOS DEPORTIVOS,282,20.0,0.070922,476403
"COMERCIO AL POR MENOR DE AVES, HUEVOS Y CAZA SIN OBRADOR",527,37.0,0.070209,472206
REPARACION DE OTROS EFECTOS PERSONALES Y ARTICULOS DE USO DOMESTICO N.C.O.P.,373,26.0,0.069705,952005
COMERCIO AL POR MENOR DE CARNICERIA-SALCHICHERIA,240,16.0,0.066667,472204
ESTABLECIMIENTOS DE VENTA DE PLATOS PREPARADOS CON OBRADOR,603,38.0,0.063018,472407
ACTIVIDADES DE LOS GIMNASIOS,558,33.0,0.059140,931008
COMERCIO AL POR MENOR DE CARNICERIA,751,44.0,0.058589,472201


There are 18 activities. I have improved the % of ones vs ceros in the Dataframe (from 2% to 6.4%). 

In [55]:
print('Number of activities with a target/total rate > 0.035: %0.0f' % (len(desc_act_fil)))
print('New target/total rate: %0.2f' % (desc_act_fil.ones.sum()/desc_act_fil.act_count.sum()))

Number of activities with a target/total rate > 0.035: 18
New target/total rate: 0.06


I save the activitiyes of interest and save this DataFrame to a new CSV: locals_sh.csv. 

In [56]:
id_epig_keep = desc_act_fil.id.values
df_locals_short = df_locals_v8[df_locals_v8.id_act_norm.isin(id_epig_keep)].copy()
cl.reset(df_locals)

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,coordenada_x_local,coordenada_y_local,desc_tipo_acceso_local,id_situacion_local,desc_situacion_local,clase_vial_acceso,desc_vial_acceso,nom_acceso,num_acceso,cal_acceso,coordenada_x_agrupacion,coordenada_y_agrup,id_agrupacion,nombre_agrupacion,id_tipo_agrup,desc_tipo_agrup,rotulo,id_seccion,desc_seccion,id_division,desc_division,id_epigrafe,desc_epigrafe,conc,desc_sit_loc_modif_19,desc_sit_loc_modif_18,desc_sit_loc_modif_17,desc_sit_loc_modif_16,desc_sit_loc_modif_15,cerrado_19,abierto_19,cerrado_18,abierto_18,cerrado_17,abierto_17,cerrado_16,abierto_16,target,desc_act_norm,id_act_norm,ab_17_19,loc_dist_act,ab_dist_act_17_19,total_loc_act,total_ab_act_17_19,loc_dist,ab_dist_17_19,loc_na_dist,ab_dist_act_17_19_rate,total_ab_act_17_19_rate,total_ab_dist_17_19_rate,total_na_dist_rate,loc_barrio_act,ab_barrio_act_17_19,loc_barrio,ab_barrio_17_19,loc_na_barrio,ab_barrio_act_17_19_rate,total_ab_barr_17_19_rate,total_na_barr_rate
0,10000003,1,CENTRO,104,JUSTICIA,"440554,59","4475338,53",Puerta Calle,1.0,Abierto,CALLE,BARCELO ...,NUM,5,,0,0,-1.0,SIN AGRUPACION,-1.0,SIN AGRUPACION,VITACA,I,HOSTELERIA,56,SERVICIOS DE COMIDAS Y BEBIDAS,561004,BAR RESTAURANTE,VITACA-BARCELO-5,Abierto,Abierto,Abierto,Abierto,Abierto,0,0,0,0,0,0,0,0,0,BAR RESTAURANTE,561004,0,726,233.0,4320,1298.0,14344,2926,2166,32.093664,30.046296,20.398773,15.100390,108,28.0,2086,528,131,25.925926,25.311601,6.279962
1,10000003,1,CENTRO,104,JUSTICIA,"440554,59","4475338,53",Puerta Calle,1.0,Abierto,CALLE,BARCELO ...,NUM,5,,0,0,-1.0,SIN AGRUPACION,-1.0,SIN AGRUPACION,VITACA,I,HOSTELERIA,56,SERVICIOS DE COMIDAS Y BEBIDAS,561006,CAFETERIA,VITACA-BARCELO-5,Abierto,Abierto,Abierto,Abierto,Abierto,0,0,0,0,0,0,0,0,0,CAFETERIA,561006,0,544,112.0,3247,935.0,14344,2926,2166,20.588235,28.795812,20.398773,15.100390,72,16.0,2086,528,131,22.222222,25.311601,6.279962
2,10000004,1,CENTRO,105,UNIVERSIDAD,"439945,6","4475591,53",Puerta Calle,1.0,Abierto,CALLE,ACUERDO ...,NUM,31,,0,0,-1.0,SIN AGRUPACION,-1.0,SIN AGRUPACION,BAR LA PARADA ...,I,HOSTELERIA,56,SERVICIOS DE COMIDAS Y BEBIDAS,561006,CAFETERIA,BAR LA PARADA-ACUERDO-31,Abierto,Abierto,Abierto,Abierto,Abierto,0,0,0,0,0,0,0,0,0,CAFETERIA,561006,0,544,112.0,3247,935.0,14344,2926,2166,20.588235,28.795812,20.398773,15.100390,101,9.0,2961,613,501,8.910891,20.702465,16.919959
3,10000013,1,CENTRO,102,EMBAJADORES,"441199,58","4473326,52",Puerta Calle,1.0,Abierto,PLAZA,EMPERADOR CARLOS V ...,NUM,8,,0,0,-1.0,SIN AGRUPACION,-1.0,SIN AGRUPACION,HOTEL MEDIODIA,I,HOSTELERIA,55,SERVICIOS DE ALOJAMIENTO,551001,HOTELES Y MOTELES CON RESTAURANTE,HOTEL MEDIODIA-EMPERADOR CARLOS V-8,Abierto,Abierto,Abierto,Abierto,Abierto,0,0,0,0,0,0,0,0,0,HOTELES Y MOTELES CON RESTAURANTE,551001,0,120,23.0,329,79.0,14344,2926,2166,19.166667,24.012158,20.398773,15.100390,12,3.0,3977,697,810,25.000000,17.525773,20.367111
4,10000044,1,CENTRO,101,PALACIO,"439722,59","4473550,53",Puerta Calle,1.0,Abierto,CALLE,HUMILLADERO ...,NUM,16,,0,0,-1.0,SIN AGRUPACION,-1.0,SIN AGRUPACION,V.M. VINOMANIA,G,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,47,"COMERCIO AL POR MENOR, EXCEPTO DE VEHICULOS DE...",472502,COMERCIO AL POR MENOR DE VINOS Y ALCOHOLES (BO...,V.M. VINOMANIA-HUMILLADERO-16,Abierto,Abierto,Abierto,Abierto,Abierto,0,0,0,0,0,0,0,0,0,OTRAS ACTIVIDADES,999999,0,44,10.0,163,34.0,14344,2926,2166,22.727273,20.858896,20.398773,15.100390,9,2.0,2193,439,415,22.222222,20.018240,18.923849
5,10000052,1,CENTRO,104,JUSTICIA,"440875,59","4475142,52",Puerta Calle,1.0,Abierto,CALLE,PELAYO ...,NUM,57,,0,0,-1.0,SIN AGRUPACION,-1.0,SIN AGRUPACION,LA ESQUINA DE SANTI,I,HOSTELERIA,56,SERVICIOS DE COMIDAS Y BEBIDAS,561005,BAR CON COCINA,LA ESQUINA DE SANTI-PELAYO-57,Abierto,Abierto,Abierto,Abierto,Abierto,0,0,0,0,0,0,0,0,0,BAR CON COCINA,561005,0,335,73.0,4572,867.0,14344,2926,2166,21.791045,18.963255,20.398773,15.100390,46,10.0,2086,528,131,21.739130,25.311601,6.279962
6,10000071,1,CENTRO,106,SOL,"440388,59

In [57]:
df_locals_short.to_csv('Data/censolocales/locals_sh.csv',index=None, header=True)

### 9) Merge with points of the same activity in a 500 m radius

I generate the number of points of the same activity in a radius via Notebook [Points_in_radius](./Points_in_radius.ipynb)   
The execution of this Notebook takes time (aprox. 50 min with my PC) so I execute it out of the Data_cleaning notebook and generate the points_in_radius.csv file that I will import and merge to generate the final dataset for the analysis

Import points_in_radius.csv and merge with locals DataFrame

In [58]:
df_points_in_radius = pd.read_csv('Data/censolocales/point_in_radius_short.csv')
df_locals_short_final = df_locals_short.merge(df_points_in_radius,how='left',on=['id_local','desc_epigrafe']).drop(['lat_y','lon_y'],axis=1).rename(columns={"lat_x": "lat", "lon_x": "lon"})
print('Final DataFrame shape:')
df_locals_short_final.shape

Final DataFrame shape:


(8893, 93)

### 10) Export to csv 
- Short file with points in radius export to locals_sh_f.csv. This will be the dataset for modeling
- Total file without points in radius export to locals_total.csv. 

In [59]:
#Short file with points in radious
df_locals_short_final.to_csv('Data/censolocales/locals_sh_f.csv',index=None, header=True)

# Total file without points in radius
df_locals_v7.to_csv('Data/censolocales/locals_total.csv',index=None, header=True)